In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn import metrics

In [2]:
df_full = pd.read_hdf('data/df_full.h5')
df_full.head()

,id,date,available,price,minimum_nights,maximum_nights,host_id,host_response_time,host_is_superhost,host_listings_count,...,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,review,month
0,2595,2022-06-04,0,225.0,30,1125,2845.0,a few days or more,0.0,6.0,...,4.72,4.62,4.76,4.79,4.86,4.41,0.0,3.0,0 11/21/2009\n1 5/28/2009\n2 ...,Jun
1,2595,2022-06-05,0,225.0,30,1125,2845.0,a few days or more,0.0,6.0,...,4.72,4.62,4.76,4.79,4.86,4.41,0.0,3.0,0 11/21/2009\n1 5/28/2009\n2 ...,Jun
2,2595,2022-06-06,0,225.0,30,1125,2845.0,a few days or more,0.0,6.0,...,4.72,4.62,4.76,4.79,4.86,4.41,0.0,3.0,0 11/21/2009\n1 5/28/2009\n2 ...,Jun
3,2595,2022-06-07,0,225.0,30,1125,2845.0,a few days or more,0.0,6.0,...,4.72,4.62,4.76,4.79,4.86,4.41,0.0,3.0,0 11/21/2009\n1 5/28/2009\n2 ...,Jun
4,2595,2022-06-08,0,225.0,30,1125,2845.0,a few days or more,0.0,6.0,...,4.72,4.62,4.76,4.79,4.86,4.41,0.0,3.0,0 11/21/2009\n1 5/28/2009\n2 ...,Jun


In [3]:
Brooklyn = df_full.copy()
Brooklyn = Brooklyn[Brooklyn['neighbourhood_group_cleansed'] == 'Brooklyn']
Brooklyn.shape

(462455, 36)

In [4]:
Brooklyn.to_hdf(r'data/Brooklyn.h5', key='stage', mode='w') #run twice

C:\Users\matth\anaconda3\lib\site-packages\pandas\core\generic.py:2703: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['host_response_time', 'host_verifications', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'property_type', 'room_type',
       'amenities', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'review', 'month'],
      dtype='object')]

  pytables.to_hdf(


As we saw in the EDA section, we should make dummies of host_response_time and room_type

In [5]:
Brooklyn_dummies = pd.get_dummies(Brooklyn, columns=['host_response_time', 'room_type'])

In [6]:
Brooklyn_dummies.corr()['price'].sort_values()

room_type_Private room                  -0.224055
longitude                               -0.130335
host_response_time_unknown              -0.032199
minimum_nights                          -0.030349
room_type_Shared room                   -0.028558
host_identity_verified                  -0.022330
has_availability                        -0.020515
number_of_reviews                       -0.018693
host_response_time_a few days or more   -0.007829
host_id                                 -0.004478
host_response_time_within a day         -0.003636
instant_bookable                         0.002072
host_has_profile_pic                     0.002449
maximum_nights                           0.002897
id                                       0.003200
host_response_time_within an hour        0.020902
host_is_superhost                        0.022441
room_type_Hotel room                     0.023467
host_response_time_within a few hours    0.023954
available                                0.027801


In [7]:
def tts_borough(df, xvars, dge):
    X = df[xvars]
    y = df['price']
    poly = PolynomialFeatures(include_bias = False, degree = dge)
    X_poly = poly.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_poly, y, random_state=42)
    return X_train, X_test, y_train, y_test

In [8]:
xvars = ['accommodates', 'beds', 'room_type_Entire home/apt', 'room_type_Private room', 'longitude', 'calculated_host_listings_count', 'latitude']
X_train, X_test, y_train, y_test = tts_borough(Brooklyn_dummies, xvars, 2)

Let's make a linear regression model


In [9]:
lr = LinearRegression()

In [10]:
cross_val_score(lr, X_train, y_train, cv = 5).mean()

0.23693995535487175

In [11]:
lr.fit(X_train, y_train)

LinearRegression()

In [12]:
prediction = lr.predict(X_test)

The function is


In [13]:
print(f'Model intercept: {lr.intercept_}')
print(f'Model coefficient values: {lr.coef_}')

Model intercept: -15406621.897594526
Model coefficient values: [-5.26436381e+04  2.78350672e+04  3.34510335e+05  3.67277968e+05
 -8.81992474e+05 -8.70833509e+03 -8.62931914e+05 -9.33417673e-01
  3.84730906e+00 -1.18270134e+02 -1.42481970e+02 -4.57576651e+02
 -5.33558791e-01  4.65937042e+02 -7.32733525e-01  1.45074871e+02
  1.39450233e+02  2.28706922e+02 -2.39224814e-01 -2.71905682e+02
  3.34510335e+05 -1.16488081e-08  8.05531778e+03  9.61180334e-01
 -1.79888183e+03  3.67277968e+05  8.77667433e+03  1.03901432e+01
 -2.09819313e+03 -9.03086270e+03 -1.72470942e+02 -1.13555209e+04
  5.80316626e-01 -9.97598825e+01  3.09877578e+02]


The R^2 value is

In [14]:
print(f'Training R2: {lr.score(X_train, y_train)}')
print(f'Testing R2: {lr.score(X_test, y_test)}')

Training R2: 0.23614506938114876
Testing R2: 0.2441888702106123


The mean squared error is


In [15]:
metrics.mean_squared_error(y_test, prediction)

34223.0373617357

And the root mean squered error is


In [16]:
np.sqrt(metrics.mean_squared_error(y_test, prediction))

184.99469549621065

How about the mean of residuals

In [17]:
residuals = y_test - prediction
np.abs(residuals).mean()

58.22872328815034

This model is not great. Let's try to improve it! We will go with Decision tree.

In [18]:
xvars = ['accommodates', 'beds', 'room_type_Entire home/apt', 'room_type_Private room', 'longitude']
X = Brooklyn_dummies[xvars]
y = Brooklyn_dummies['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [19]:
#I tweeked this until I got the best result
dt = DecisionTreeRegressor(random_state=42)

In [20]:
dt.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [21]:
prediction = dt.predict(X_test)

In [22]:
print(f'Score on training set: {dt.score(X_train, y_train)}')
print(f'Score on testing set: {dt.score(X_test, y_test)}')

Score on training set: 0.9857853579984303
Score on testing set: 0.9844286292511029


In [23]:
metrics.mean_squared_error(y_test, prediction)

705.0698010512767

In [24]:
np.sqrt(metrics.mean_squared_error(y_test, prediction))

26.55315049200898

In [25]:
residuals = y_test - prediction
np.abs(residuals).mean()

7.699352561114326

This model is very good! Let's try the bagging model.


In [26]:
xvars = ['accommodates', 'beds', 'room_type_Entire home/apt', 'room_type_Private room', 'longitude']
X = Brooklyn_dummies[xvars]
y = Brooklyn_dummies['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [27]:
#From lesson 6.02
bag = BaggingRegressor(random_state = 42)
bag.fit(X_train, y_train)

BaggingRegressor(random_state=42)

In [28]:
prediction = bag.predict(X_test)

In [29]:
print(f'Score on training set: {bag.score(X_train, y_train)}')
print(f'Score on testing set: {bag.score(X_test, y_test)}')

Score on training set: 0.9857823912653305
Score on testing set: 0.9844264832704798


In [30]:
metrics.mean_squared_error(y_test, prediction)

705.1669707966654

In [31]:
np.sqrt(metrics.mean_squared_error(y_test, prediction))

26.55498015056056

In [32]:
residuals = y_test - prediction
np.abs(residuals).mean()

7.698506449112763

___

RandomForest:

In [33]:
xvars = ['accommodates', 'beds', 'room_type_Entire home/apt', 'room_type_Private room', 'longitude']
X = Brooklyn_dummies[xvars]
y = Brooklyn_dummies['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [34]:
rf = RandomForestRegressor(n_estimators = 100)

In [35]:
rf.fit(X_train,y_train)

RandomForestRegressor()

In [36]:
prediction = rf.predict(X_test)

In [37]:
print(f'Score on training set: {rf.score(X_train, y_train)}')
print(f'Score on testing set: {rf.score(X_test, y_test)}')

Score on training set: 0.9857849647988912
Score on testing set: 0.9844215131757307


In [38]:
metrics.mean_squared_error(y_test, prediction)

705.392016091163

In [39]:
np.sqrt(metrics.mean_squared_error(y_test, prediction))

26.55921715885397

In [40]:
residuals = y_test - prediction
np.abs(residuals).mean()

7.699553443069883

ExtraTrees:

In [41]:
xvars = ['accommodates', 'beds', 'room_type_Entire home/apt', 'room_type_Private room', 'longitude']
X = Brooklyn_dummies[xvars]
y = Brooklyn_dummies['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [42]:
et = ExtraTreesRegressor(n_estimators = 100)

In [43]:
et.fit(X_train, y_train)

ExtraTreesRegressor()

In [44]:
prediction = et.predict(X_test)

In [45]:
print(f'Score on training set: {et.score(X_train, y_train)}')
print(f'Score on testing set: {et.score(X_test, y_test)}')

Score on training set: 0.9857853579984303
Score on testing set: 0.9844286292511029


In [46]:
metrics.mean_squared_error(y_test, prediction)

705.0698010512764

In [47]:
np.sqrt(metrics.mean_squared_error(y_test, prediction))

26.553150492008974

In [48]:
residuals = y_test - prediction
np.abs(residuals).mean()

7.699352561114326

All the models, except for linear regression, are good models and basically the same. Any one can be used. So I'll go with the decision tree for the streamlit, since that's what I'll use for the other ones.
